In [1]:
import os
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from serpapi import GoogleSearch
from pydantic import BaseModel

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [3]:
class OutputStructure(BaseModel):
    title: str
    description: str
    rating: str
    highlighted_review: str


In [4]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)

In [ ]:
@tool
def attraction_finding_tool(place_name: str) -> list[dict]:
    """  
    Find popular attractions in a city or country.

    Args:
        place_name (str): Name of the city or country.

    Returns:
        list[dict]: List of up to 5 attractions with details:
            - title (str)
            - description (str)
            - rating (str/float)
            - highlighted_review (str)
        If no attractions found, returns a list with a dict containing an "error" key.
    """
    params = {
    "engine": "tripadvisor",
    "q": place_name,
    "ssrc": "A",
    "api_key": ""
    }

    search = GoogleSearch(params)
    try:

        results = search.get_dict()
        attraction_places = results.get("places", [])
    except Exception as e:
        return [{"error": f"There are no attraction places found in {place_name}."}]
    if not attraction_places:
        return []

    attraction_details = []

    for num in range(5):
        attraction = attraction_places[num]
        attraction_details.append(
            {
                "title": attraction.get('title', []),
                "description": attraction.get('description', []),
                "rating": attraction.get('rating', []),
                "highlighted_review": attraction.get('highlighted_review', [])
            }
        )
    return attraction_details
    


In [ ]:
import os
from serpapi import GoogleSearch

@tool
def restaurant_finding_tool(place_name: str) -> list[dict]:
    """
    Tool Name: Restaurant Finder

    Purpose:
        Find top restaurants in a specified city using TripAdvisor.

    Input:
        place_name (str): Name of the city or country.

    Output:
        List of dictionaries (up to 5 items), each containing:
            - title (str): Name of the restaurant
            - description (str): Short description
            - rating (str/float): Average user rating
            - highlighted_review (str): Notable user review
        If no restaurants are found, returns a list with a dictionary containing an "error" key.

    Usage Instructions for AI:
        Use this tool when a user asks about restaurants or places to eat in a specific city.
    """
    
    params = {
        "engine": "tripadvisor",
        "q": place_name,
        "ssrc": "r",  # 'r' for restaurants
        "api_key": ""
    }

    search = GoogleSearch(params)

    try:
        results = search.get_dict()
        places = results.get("places", [])
    except Exception as e:
        return [{"error": f"Error fetching restaurants: {e}"}]

    if not places:
        return [{"error": f"No restaurants found in {place_name}."}]

    top_places = places[:5]

    restaurant_details = []
    for place in top_places:
        restaurant_details.append({
            "title": place.get("title", ""),
            "description": place.get("description", ""),
            "rating": place.get("rating", ""),
            "highlighted_review": place.get("highlighted_review", "")
        })

    return restaurant_details


In [ ]:
import os
from serpapi import GoogleSearch

@tool
def hotel_finding_tool(place_name: str) -> list[dict]:
    """
    Tool Name: Hotel Finder

    Purpose:
        Find top hotels in a specified city using TripAdvisor.

    Input:
        place_name (str): Name of the city or country.

    Output:
        List of dictionaries (up to 5 items), each containing:
            - title (str): Name of the hotel
            - description (str): Short description
            - rating (str/float): Average user rating
            - highlighted_review (str): Notable user review
        If no hotels are found, returns a list with a dictionary containing an "error" key.

    Usage Instructions for AI:
        Use this tool when a user asks about hotels, accommodations, or places to stay in a specific city.
    """
    
    params = {
        "engine": "tripadvisor",
        "q": place_name,
        "ssrc": "h",  # 'h' for hotels
        "api_key": ""
    }

    search = GoogleSearch(params)

    try:
        results = search.get_dict()
        places = results.get("places", [])
    except Exception as e:
        return [{"error": f"Error fetching hotels: {e}"}]

    if not places:
        return [{"error": f"No hotels found in {place_name}."}]

    top_places = places[:5]

    hotel_details = []
    for place in top_places:
        hotel_details.append({
            "title": place.get("title", ""),
            "description": place.get("description", ""),
            "rating": place.get("rating", ""),
            "highlighted_review": place.get("highlighted_review", "")
        })

    return hotel_details


In [8]:
trip_advisor_prompt = """You are CityDiscoveryAdvisor. Find attractions, hotels, and restaurants for cities using TripAdvisor tools.

TOOLS:
- attraction_finding_tool(place_name): Top 5 attractions
- hotel_finding_tool(place_name): Top 5 hotels
- restaurant_finding_tool(place_name): Top 5 restaurants

USAGE:
- "things to do" → attractions
- "where to stay" → hotels
- "where to eat" → restaurants
- General city query → use all 3 tools

RULES:
1. Only use tool outputs, never fabricate data
2. If tool returns error, report: "No {category} found for {place}"
3. Present all results (up to 5 per category)

FORMAT:
**{Category} in {City}:**
1. {Name} ({Rating}) - {Description}
   "{Highlighted Review}"

**Note:** {Any errors or limitations}
"""


trip_advisor_agent = create_agent(model = model, system_prompt = trip_advisor_prompt, tools = [attraction_finding_tool, hotel_finding_tool, restaurant_finding_tool])

In [10]:
ans = trip_advisor_agent.invoke({"messages": "I'm going to Islamabad for 3 days, tell me what I eat ? where I should live and where I go and enjoy"})
print(ans['messages'][-1].content[0]['text'])

**Restaurants in Islamabad:**
1. Kabul Restaurant (4) - Kabul restaurant started in 1983, and a good restaurant with a pleasing ambiance and quality and delicious food. Being one of the most prominent restaurants in F7 Markaz Islamabad provides you with a wide range of appetizers, and delicious Afghani food.
   "I had the pleasure of dining at one of the best Restaurant in Islamabad"
2. The Royal Elephant (4.3) - Offers Thai cusine. Two Private Rooms for Fine Dining one with the capacity of 10-persons and other with the capacity of 06-persons.
   "We had a good time once again at the Royal Elephant at the Marriott Islamabad."
3. chaayé khana (3.9) - The idea for Chaaye Khana was born from our love of tea. A cup of doodh patti in the evening, the Peshawari Kehwa after dinner, and of course, the morning cup of special tea to wake us up. This is what our culture is about, this is the benchmark of our hospitality. We went in search of the best tea leaves we could find locally and internati